In [2]:
import sys
import os
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
import random as rn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import array, savetxt, asarray, save
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, Dropout
from keras.utils import np_utils
#from google.colab import drive
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers import GRU
#drive.mount('/content/drive')
import keras_tuner
from keras_tuner import HyperModel

DATA_PATH = "data/data.txt"
NEW_DATA = "data/processed_data.txt"
TRAIN_DATA = "data/train_data.txt"
TEST_DATA = "data/test_data.txt"
VAL_DATA = "data/val_data.txt"

Filter Dataset Into Smaller Segments

In [4]:
count=0
new = open(NEW_DATA, "w")
for line in open(DATA_PATH, "r"):
    count += 1
    if 30 < len(line) < 50 and count < 300000:
        if ('T' not in line) and ('V' not in line) and ('g' not in line) and ('L' not in line) and ('8' not in line):
            new.write(line)
file_new = np.array(list(open(NEW_DATA)))
print(file_new.shape)


(123653,)


Find Maximum Sequence Length

In [6]:
file = open(NEW_DATA)
max_seq_len = int(len(max(file,key=len)))
print ("Max Sequence Length: ", max_seq_len)

Max Sequence Length:  49


Define Functions

In [7]:

def read(fileName): 
        fileObj = open(fileName, "r") 
        words = fileObj.read().splitlines() 
        fileObj.close() 
        return words 
def padFile(fileName): 
  temp = read(fileName) 
  preprocessed_pad_text = [['?'] + list(i) for i in temp] 
  print("Sample 1: ", preprocessed_pad_text[0]) 
  print("Sample 2: ", preprocessed_pad_text[1]) 
  print("Sample 3: ", preprocessed_pad_text[2]) 
  print("Sample 4: ", preprocessed_pad_text[3]) 
  print("Sample 5: ", preprocessed_pad_text[4]) 
  padded_text = pad_sequences(preprocessed_pad_text, dtype=object, maxlen=max_seq_len+1, padding="post", value="!") 
  # front_pad = [['?'] + i for i in padded_text] 
  #print("Padded Text Arrays: ", padded_text) 
  var = ["".join(i) for i in padded_text] 
  print("Padded Strings: ", var[0:5]) 
  #print(var) 
  # with open('/content/drive/MyDrive/CS230/padded_processed_data.txt', "w") as output: 
  #   for x in var: 
  #     output.write(y) 
  # np.array(var) 
  # np.savetxt('/content/drive/MyDrive/CS230/padded_processed_data.txt', var, fmt ='%s', newline='') 
  return var 
var = padFile(NEW_DATA) 
var = np.array(var) 


Sample 1:  ['?', 'C', 'O', 'c', '1', 'c', 'c', '2', 'n', 'c', '(', 'n', 'c', '(', 'N', ')', 'c', '2', 'c', 'c', '1', 'O', 'C', ')', 'N', '3', 'C', 'C', 'N', '(', 'C', 'C', '3', ')', 'C', '(', '=', 'O', ')', 'c', '4', 'o', 'c', 'c', 'c', '4']
Sample 2:  ['?', 'C', 'C', 'N', '1', 'C', '=', 'C', '(', 'C', '(', '=', 'O', ')', 'O', ')', 'C', '(', '=', 'O', ')', 'c', '2', 'c', 'c', 'c', '(', 'C', ')', 'n', 'c', '1', '2']
Sample 3:  ['?', 'C', 'C', 'N', '1', 'C', '=', 'C', '(', 'C', '(', '=', 'O', ')', 'O', ')', 'C', '(', '=', 'O', ')', 'c', '2', 'c', 'c', '(', 'F', ')', 'c', '(', 'c', 'c', '1', '2', ')', 'N', '3', 'C', 'C', 'N', 'C', 'C', '3']
Sample 4:  ['?', 'C', 'N', '1', 'C', '(', '=', 'O', ')', 'C', 'N', '=', 'C', '(', 'c', '2', 'c', 'c', 'c', 'c', 'c', '2', ')', 'c', '3', 'c', 'c', '(', 'C', 'l', ')', 'c', 'c', 'c', '1', '3']
Sample 5:  ['?', 'O', 'C', '1', '=', 'N', 'C', '(', '=', 'O', ')', 'C', '(', 'N', '1', ')', '(', 'c', '2', 'c', 'c', 'c', 'c', 'c', '2', ')', 'c', '3', 'c', 'c', 

Load & Save Datasets

In [8]:
#init random seed
seed = 1
np.random.seed(seed)
#split data into train/test
full_train, test = train_test_split(np.array(var), test_size=0.2, random_state=seed)
# full_train, test = train_test_split(np.array(var), test_size=0.25, random_state=seed)

split full train set into smaller train set and validation (dev) set

In [9]:
#np.savetxt('/content/drive/MyDrive/CS230/full_train_data.txt', full_train, fmt ='%s', newline='')
train, val = train_test_split(np.array(full_train), test_size=0.10, random_state=seed)
np.savetxt(TRAIN_DATA, train, fmt ='%s', newline='')
np.savetxt(TEST_DATA, test, fmt ='%s', newline='')
np.savetxt(VAL_DATA, val, fmt='%s', newline='')
print("Sample:", train[seed])
print("Train:", train.shape)
print("Validation:", val.shape)
print("Test:", test.shape)

Sample: ?COc1cccc(CNCCCCCCNCCSSCCNCCCCCCNCc2ccccc2OC)c1!!!
Train: (89029,)
Validation: (9893,)
Test: (24731,)


Train Data Processing

In [10]:
#Load Data (optional: only if previous cells are not run and data is saved already) 
# train = pd.read_fwf('/content/drive/MyDrive/CS230/train_data.txt') 
#Concatenate Data 
def concatenate(data): 
    #res = '' 
    print(len(data)) 
    #for count, word in enumerate(data): 
        #if count %1000 == 0: 
            #print(count) 
            #pass 
        # print(word) 
        #res += word 
    res = ''.join(data) 
    return res 
train = concatenate(train) 
print(train[0:100]) 





89029
?N[C@@H](Cc1c[nH]c2ccccc12)C(=O)OCc3ccccc3!!!!!!!!?COc1cccc(CNCCCCCCNCCSSCCNCCCCCCNCc2ccccc2OC)c1!!!


In [11]:
#Tokenize Data 
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, char_level=True, lower=False) 
tokenizer.fit_on_texts(train) 
new_train = tokenizer.texts_to_sequences(train) 
print(new_train[0:100]) 
print(train[0:100]) 

[[12], [9], [15], [3], [17], [17], [18], [16], [4], [3], [1], [7], [1], [15], [13], [18], [16], [1], [8], [1], [1], [1], [1], [1], [7], [8], [5], [3], [4], [10], [6], [5], [6], [3], [1], [11], [1], [1], [1], [1], [1], [11], [2], [2], [2], [2], [2], [2], [2], [2], [12], [3], [6], [1], [7], [1], [1], [1], [1], [4], [3], [9], [3], [3], [3], [3], [3], [3], [9], [3], [3], [22], [22], [3], [3], [9], [3], [3], [3], [3], [3], [3], [9], [3], [1], [8], [1], [1], [1], [1], [1], [8], [6], [3], [5], [1], [7], [2], [2], [2]]
?N[C@@H](Cc1c[nH]c2ccccc12)C(=O)OCc3ccccc3!!!!!!!!?COc1cccc(CNCCCCCCNCCSSCCNCCCCCCNCc2ccccc2OC)c1!!!


In [12]:
#Print Data Breakdown 
n_chars = len(train) 
n_vocab = len(list(set(train))) 
print("# of Unique Characters:", n_chars) 
print("# of Total Characters:", n_vocab) 
n_chars = len(new_train) 
n_vocab = len(set(train)) 
print("# of Unique Characters:", n_chars) 
print("# of Total Characters:", n_vocab) 
#N-Grams Sequence 
seqLen = 15 
stepSize = 1 
input_chars = [] 
next_char = [] 
for i in range(0, len(new_train) - seqLen, stepSize): 
  input_chars.append(new_train[i : i + seqLen]) 
  next_char.append(new_train[i + seqLen]) 
for i in range(5): 
  print("Input Sequence:", input_chars[i]) 
  print("Next Character Prediction:", next_char[i]) 
#Assemble Train Datasets 
x_train = np.array(input_chars) 
x_train.flatten() 
y_train = np.array(next_char) 
y_train_2 = np_utils.to_categorical(y_train) 
print(x_train.shape) 
print(y_train.shape) 
print(x_train[0:5]) 
print(y_train[0:5]) 

# of Unique Characters: 4451450
# of Total Characters: 42
# of Unique Characters: 4451450
# of Total Characters: 42
Input Sequence: [[12], [9], [15], [3], [17], [17], [18], [16], [4], [3], [1], [7], [1], [15], [13]]
Next Character Prediction: [18]
Input Sequence: [[9], [15], [3], [17], [17], [18], [16], [4], [3], [1], [7], [1], [15], [13], [18]]
Next Character Prediction: [16]
Input Sequence: [[15], [3], [17], [17], [18], [16], [4], [3], [1], [7], [1], [15], [13], [18], [16]]
Next Character Prediction: [1]
Input Sequence: [[3], [17], [17], [18], [16], [4], [3], [1], [7], [1], [15], [13], [18], [16], [1]]
Next Character Prediction: [8]
Input Sequence: [[17], [17], [18], [16], [4], [3], [1], [7], [1], [15], [13], [18], [16], [1], [8]]
Next Character Prediction: [1]
(4451435, 15, 1)
(4451435, 1)
[[[12]
  [ 9]
  [15]
  [ 3]
  [17]
  [17]
  [18]
  [16]
  [ 4]
  [ 3]
  [ 1]
  [ 7]
  [ 1]
  [15]
  [13]]

 [[ 9]
  [15]
  [ 3]
  [17]
  [17]
  [18]
  [16]
  [ 4]
  [ 3]
  [ 1]
  [ 7]
  [ 1]
  [15

In [13]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        ind1 = 0
        ind2 = 14
        temp = []
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = np.array(token_list)
        for token in token_list:
            temp.append(token)
        
        #token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        token_list=np.array([temp])
        token_list = token_list[ind1:ind2+1]
        print(token_list.shape)
        ind1 += 1
        
        ind2 += 1
        print(token_list)
        #np.reshape(token_list, (15,1))
        predicted = model.predict(token_list, verbose=0)
        #print(predicted)
        ind=np.argmax(predicted)
        print(ind)
        print(f"Shape: {predicted.shape}")
        output_word = ""
        for word,index in tokenizer.word_index.items():
            '''
            print(tokenizer.word_index.items())
            print(word)
            print(ind)
            print(index)
            '''
            
            if index == ind:
                output_word = word
                break
        seed_text += output_word
    return seed_text.title()

In [16]:
#seed ='!!!!!!!!!!!!!!?'
seed = '?CN(C)C1=NC(=NC'
model = keras.models.load_model('baseline_lstm.h5')
print(model.summary())
result = generate_text(seed, 35, model, 50)
print(result)
test = model.predict([[3], [20], [1], [7], [1], [1], [1], [1], [8], [1], [1], [4], [3], [3], [11]])
test = np.argmax(test)
print(test)
print(tokenizer.word_index.items())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 dropout (Dropout)           (None, 15, 32)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 43)                2795      
                                                                 
Total params: 31,979
Trainable params: 31,979
Non-trainable params: 0
_________________________________________________________________
None
(1, 15)
[[12  3  9  4  3  5  3  7 10  9  3  4 1

In [17]:
#Load Data (optional: only if previous cells are not run and data is saved already) 
#val = pd.read_fwf('/content/drive/MyDrive/CS230/val_data.txt')

In [18]:

''' 
#Concatenate Data 
def concatenate(data): 
    #res = '' 
    #for count, word in enumerate(data): 
     #   if count %100 == 0: 
      #      print(count) 
     
      # print(word) 
      #  res += word 
    res = ''.join(data) 
    return res 
val = concatenate(val) 
print(val[0:100]) 
#Tokenize Data 
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, char_level=True, lower=False) 
tokenizer.fit_on_texts(val) 
new_val = tokenizer.texts_to_sequences(val) 
print(new_val[0:100]) 
print(val[0:100]) 
#Print Data Breakdown 
n_chars = len(val) 
n_vocab = len(list(set(val))) 
print("# of Unique Characters:", n_chars) 
print("# of Total Characters:", n_vocab) 
n_chars = len(new_val) 
n_vocab = len(set(val)) 
print("# of Unique Characters:", n_chars) 
print("# of Total Characters:", n_vocab) 
print("Difference", set(train).difference(set(val))) 
#N-Grams Sequence 
seqLen = 15 
stepSize = 1 
input_chars = [] 
next_char = [] 
for i in range(0, len(new_val) - seqLen, stepSize): 
  input_chars.append(new_val[i : i + seqLen]) 
  next_char.append(new_val[i + seqLen]) 
#print(set(next_char.items)) 
for i in range(5): 
  print("Input Sequence:", input_chars[i]) 
  print("Next Character Prediction:", next_char[i]) 
#Assemble Validation Datasets 
x_val = np.array(input_chars) 
x_val.flatten() 
y_val = np.array(next_char) 
y_val_2 = np_utils.to_categorical(y_val) 
print(y_val_2) 
print(y_train_2) 
#score = model.evaluate(x_val, y_val_2) 
#print(score[1]) 
#print(result) 
'''


' \n#Concatenate Data \ndef concatenate(data): \n    #res = \'\' \n    #for count, word in enumerate(data): \n     #   if count %100 == 0: \n      #      print(count) \n     \n      # print(word) \n      #  res += word \n    res = \'\'.join(data) \n    return res \nval = concatenate(val) \nprint(val[0:100]) \n#Tokenize Data \ntokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, char_level=True, lower=False) \ntokenizer.fit_on_texts(val) \nnew_val = tokenizer.texts_to_sequences(val) \nprint(new_val[0:100]) \nprint(val[0:100]) \n#Print Data Breakdown \nn_chars = len(val) \nn_vocab = len(list(set(val))) \nprint("# of Unique Characters:", n_chars) \nprint("# of Total Characters:", n_vocab) \nn_chars = len(new_val) \nn_vocab = len(set(val)) \nprint("# of Unique Characters:", n_chars) \nprint("# of Total Characters:", n_vocab) \nprint("Difference", set(train).difference(set(val))) \n#N-Grams Sequence \nseqLen = 15 \nstepSize = 1 \ninput_chars = [] \nnext_char = [] \nfor i in ran

Model Definition

In [ ]:
baselineLSTM = Sequential()
# baselineLSTM.add(LSTM(units = 64, input_shape = (x_train.shape[1:])))
baselineLSTM.add(LSTM(units = 32, return_sequences= True, input_shape = (x_train.shape[1:])))
baselineLSTM.add(Dropout(0.2))
baselineLSTM.add(LSTM(units = 64))
baselineLSTM.add(Dropout(0.2))
baselineLSTM.add(Dense(units = 43, activation='softmax'))
baselineLSTM.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(baselineLSTM.summary())

Train LSTM Model

In [19]:
''' 
early = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) 
#print(x_val.shape) 
print(x_train.shape) 
print(y_val_2.shape) 
print(y_train_2.shape) 
baselineLSTM.fit(x_train, y_train_2, epochs = 50, batch_size = 128, validation_data=(x_val, y_val_2), verbose=1, callbacks = [early]) 
#INSERT PATH HERE TO SAVE MODEL TO 
path = 'baseline_lstm.h5' 
baselineLSTM.save(path) 
#Load Model 
savedBaselineLSTM = load_model(path) 
''' 


" \nearly = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) \n#print(x_val.shape) \nprint(x_train.shape) \nprint(y_val_2.shape) \nprint(y_train_2.shape) \nbaselineLSTM.fit(x_train, y_train_2, epochs = 50, batch_size = 128, validation_data=(x_val, y_val_2), verbose=1, callbacks = [early]) \n#INSERT PATH HERE TO SAVE MODEL TO \npath = 'baseline_lstm.h5' \nbaselineLSTM.save(path) \n#Load Model \nsavedBaselineLSTM = load_model(path) \n"

In [20]:
'''
hybrid = Sequential() 
hybrid.add(LSTM(units = 16, return_sequences= True, input_shape = (x_train.shape[1:]))) 
hybrid.add(Dropout(0.2)) 
hybrid.add(GRU(units = 32, return_sequences=True)) 
hybrid.add(Dropout(0.2)) 
hybrid.add(LSTM(units = 32)) 
hybrid.add(Dropout(0.2)) 
hybrid.add(Dense(units = 43, activation='softmax')) 
hybrid.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
print(hybrid.summary()) 
early = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) 
hybrid.fit(x_train, y_train_2, epochs = 1, batch_size = 128, validation_data=(x_val, y_val_2), callbacks = [early]) 
path = 'hybrid.h5' 
hybrid.save(path) 
''' 


"\nhybrid = Sequential() \nhybrid.add(LSTM(units = 16, return_sequences= True, input_shape = (x_train.shape[1:]))) \nhybrid.add(Dropout(0.2)) \nhybrid.add(GRU(units = 32, return_sequences=True)) \nhybrid.add(Dropout(0.2)) \nhybrid.add(LSTM(units = 32)) \nhybrid.add(Dropout(0.2)) \nhybrid.add(Dense(units = 43, activation='softmax')) \nhybrid.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) \nprint(hybrid.summary()) \nearly = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) \nhybrid.fit(x_train, y_train_2, epochs = 1, batch_size = 128, validation_data=(x_val, y_val_2), callbacks = [early]) \npath = 'hybrid.h5' \nhybrid.save(path) \n"

In [21]:
''' 
class Model(HyperModel): 
  def build(self, hp): 
    units1 = hp.Int('units1', min_value=32, max_value=512, step=32) 
    dropout1 = hp.Float(name="dropout", min_value=0.0, max_value=0.3, step=0.05) 
    units2 = hp.Int('units2', min_value=32, max_value=512, step=32) 
    dropout2 = hp.Float(name="dropout", min_value=0.0, max_value=0.3, step=0.05) 
    units3 = hp.Int('units3', min_value=32, max_value=512, step=32) 
    dropout3 = hp.Float(name="dropout", min_value=0.0, max_value=0.3, step=0.05) 
    units4 = hp.Int('units2', min_value=32, max_value=512, step=32) 
    dropout4 = hp.Float(name="dropout", min_value=0.0, max_value=0.3, step=0.05) 
    model = Sequential() 
    model.add(LSTM(units = units1, return_sequences= True, input_shape = (x_train.shape[1:]))) 
    model.add(Dropout(dropout1)) 
    model.add(LSTM(units = units2, return_sequences=True)) 
    model.add(Dropout(dropout2)) 
    model.add(LSTM(units = units3, return_sequences = True)) 
    model.add(Dropout(dropout3)) 
    model.add(LSTM(units = units4)) 
    model.add(Dropout(dropout4)) 
    model.add(Dense(units=43, activation = 'softmax')) 
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
    return model 
hypermodel = Model() 
tuner = keras_tuner.BayesianOptimization( 
                        hypermodel=hypermodel, 
                        objective = "val_accuracy", 
                        max_trials =3, 
                        overwrite=True, 
                        directory='BO_search_dir', 
                        project_name='better_lstm') 
tuner.search(x_train, y_train_2, epochs=3, validation_data=(x_val, y_val_2)) 
best_model = tuner.get_best_models()[0] 
best_model.build(input_shape= (x_train.shape[1:])) 
best_model.summary() 
print(tuner.results_summary()) 
''' 


' \nclass Model(HyperModel): \n  def build(self, hp): \n    units1 = hp.Int(\'units1\', min_value=32, max_value=512, step=32) \n    dropout1 = hp.Float(name="dropout", min_value=0.0, max_value=0.3, step=0.05) \n    units2 = hp.Int(\'units2\', min_value=32, max_value=512, step=32) \n    dropout2 = hp.Float(name="dropout", min_value=0.0, max_value=0.3, step=0.05) \n    units3 = hp.Int(\'units3\', min_value=32, max_value=512, step=32) \n    dropout3 = hp.Float(name="dropout", min_value=0.0, max_value=0.3, step=0.05) \n    units4 = hp.Int(\'units2\', min_value=32, max_value=512, step=32) \n    dropout4 = hp.Float(name="dropout", min_value=0.0, max_value=0.3, step=0.05) \n    model = Sequential() \n    model.add(LSTM(units = units1, return_sequences= True, input_shape = (x_train.shape[1:]))) \n    model.add(Dropout(dropout1)) \n    model.add(LSTM(units = units2, return_sequences=True)) \n    model.add(Dropout(dropout2)) \n    model.add(LSTM(units = units3, return_sequences = True)) \n    mo